In [9]:
from tensorflow.keras import layers, models, Input

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFont, ImageDraw
import random
from keras.preprocessing.image import load_img, img_to_array

data_directory = 'sketch/small'
TARGET_SIZE = (128,128)
EPOCHS = 10
BATCH_SIZE = 32

In [4]:
def create_pairs(directory):
    #directory is the path of the dataset (main folder)
    #folders are classes in dataset
    pairs = []
    labels = []
    
    folders = os.listdir(directory)
    for folder_name in folders:
        folder_path = os.path.join(directory, folder_name)
        
        if os.path.isdir(folder_path):
            images = os.listdir(folder_path)
            
            folder_length = len(images)
            for i in range(folder_length):
                for j in range(folder_length):
                    if i != j:
                        image_path = os.path.join(folder_path, images[i])
                        pairs.append([image_path, os.path.join(folder_path, images[j])])
                        labels.append(1)#positive pairs
                        
                        dif_folder = random.choice([x for x in folders if x != folder_name])
                        dif_folder_path = os.path.join(directory, dif_folder)
                        dif_image_path = os.path.join(dif_folder_path, random.choice(os.listdir(dif_folder_path)))

                        pairs.append([image_path, dif_image_path])
                        labels.append(0)#negative pairs
                        
    return np.array(pairs), np.array(labels) 

In [ ]:
def load_images(pairs, target_size):
    images = []
    
    for pair in pairs:
        
    

In [ ]:
def load_split_preprocess(directory):
    X, y = create_pairs(directory)
    
    

In [7]:
def create_siamese(input_shape=TARGET_SIZE):
    # Define the base CNN architecture
    base_model = models.Sequential()
    base_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    base_model.add(layers.MaxPooling2D((2, 2)))
    base_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    base_model.add(layers.MaxPooling2D((2, 2)))
    base_model.add(layers.Flatten())
    base_model.add(layers.Dense(128, activation='relu'))

    # Define the left input and right input
    left_input = layers.Input(shape=input_shape)
    right_input = layers.Input(shape=input_shape)

    # Encode each input using the base CNN
    encoded_left = base_model(left_input)
    encoded_right = base_model(right_input)

    # Compute the L1 distance between the encoded outputs
    l1_distance = layers.Lambda(lambda tensors: abs(tensors[0] - tensors[1]))([encoded_left, encoded_right])

    # Add a dense layer with sigmoid activation to output similarity score
    prediction = layers.Dense(1, activation='sigmoid')(l1_distance)

    # Connect the inputs with the prediction
    siamese_model = models.Model(inputs=[left_input, right_input], outputs=prediction)

    return siamese_model

In [8]:
siamese_model = create_siamese()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\vlns\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
